<!--NAVIGATION-->
< [Solving Problems Numerically](Lecture.01-Solving-Problems-Numerically.ipynb) | [Contents](Index.ipynb) | [Bicycle Racing: The Effect of Air Resistance](Lecture.03-Bicyle-Racing.ipynb) >

# Numerical Programming
## Lecture 2

Let's begin with our radioactive decay process and its numerical solution. Run the following code block and think careful about the various steps.

* Are there any lines of code that you have questions about?

* Does the code and plot work on your computer?

* What lines are the code for the algorithm at the heart of this numerical problem?
(pressing the 'l' key will toggle line numbers on and off in Jupyter)

After about 5-10 minutes when you've had time to study the code yourself, we will discuss it as a class.

In [ ]:
# 1D radioactive decay

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

# initial number of uranium atoms
N_uranium_initial = 1000

# decay constant of 238 U (expressed as 'per year')
k = 4.916e-18*365*24*60*60

# simulation to maximum number of years
tmax = 10.0e9

# time step in years
dt = 100e6

# calculate an array for all times
t = np.arange(0, tmax, dt)

# determine total number of timesteps needed
M = len(t)

# initializes N_uranium, an array of dimension M of all zeros
N_uranium = np.zeros(M)

# the initial condition, first entry in the array N_uranium is N_uranium_initial
N_uranium[0] = N_uranium_initial

# loop over the timesteps and calculate the numerical solution
for i in range(1, M):
    N_uranium[i] = N_uranium[i-1] - (k * N_uranium[i-1])*dt

# For comparison, calculate the analytical solution
N_analytical = N_uranium_initial * np.exp(-k * t)

# create a new figure and axes for plotting
fig, axes = plt.subplots()

# Plot the numerical solution in red circles
plt.plot(t, N_uranium, 'r-', label = 'Numerical'); 
# Plot the numerical solution with blue line
plt.plot(t, N_analytical, 'b-', label = 'Analytical'); 
# label axes
plt.xlabel('Time in years')
plt.ylabel('Number of atoms')

# axes scales
plt.xlim(0, tmax)
plt.ylim(100, 1000)
plt.legend()

plt.show()

## Questions, Comments?



## Good programming style

*Giordano 1.6*

You can think of a program of a sequence of steps that you are telling the computer to complete.  While programming is necessarily personal and individual, there are are general guidelines to keep in mind. 

1\. **Program structure**.  Use subroutines or functions to organize the major taks and make the program readable and understandable.  Use these functions to perform any taks that take more than a few lines of code, or that are required repeatedly. Often, a *main* function is used to provide an outline of the program. 

2\. **Use descriptive names**.  Choose the names of variables and functions according to the problem at hand. Descriptive names make a program easier to understand, as they act as built-in comment statements.

3\. **Use comment statements**.  Include comment statements to explain program logic and describe variables.  A short function that uses descriptive variables names should not need a large number of comment statements.

4\. **Sacrifice everything for clarity**. This is a bit overstated, but not much! It is often tempting to write a critical piece of code in a very compact or terse manner in the beliefe that this will make the program run faster. This compactness always comes at the prices of clairity and readability.  It is always better to take a few more lines, or a few more variables to a job, if it makes the more understandable.  Execution speed is rarely a critical issue especially in the context of the time and effort to write and read code by a human!

5\. **Take the time to make the graphics presentable**. In almost all cases, numerical results should be presented graphically. The axes should be labeled clearly (including units, where appropriate) and parameter values given directly on the graph.

### Python functions

The radioactive decay code above follows most of the advice given above except the rules about **Program structure**.  To be able to do this we need to introduce the idea of a subroutine in a programming language. Subroutines in Python are called **functions** which are similar, but not quite the same, as mathematical functions.

Functions are made using the **def** keyword.

In [ ]:
# define a function
def forty_two():
    return 42

In [ ]:
# call a function
forty_two()

In [ ]:
# call the function, and print the result
print(forty_two())

In [ ]:
# call the function, assign the result 
# to x, and print x
x = forty_two()
print(x)

#### Function arguments

Functions can take inputs (known as arguments) and **return** output.

In [ ]:
# functions can take both positional and keyword arguments
def line(x, a=1, b=0):
    return a * x + b

In [ ]:
line(42)            # no keyword args, returns 1*42 + 0

In [ ]:
line(42, 2)         # a=2, returns 84

In [ ]:
line(42, b=10)      # a=1, b=10, returns 52

In [ ]:
line(42, b=10, a=2) # returns 94

#### Multiple return values

Sometimes we want a Python function to return more than one value. The way to do this is use a *tuple*.  A tuple is a immutable sequence of items separated by commas and surrounded by parentheses. In Python it is common to use tuples (a generalized way of saying double, triples, quadruples) to pass data in and out of functions.

In [ ]:
def momentum_energy(m, v):
    p = m * v
    e = 0.5 * m * v**2
    return (p, e)

In [ ]:
# returns a tuple
p_e = momentum_energy(42.0, 65.0)
print(p_e)

In [ ]:
# unpacks the tuple
mom, eng = momentum_energy(42.0, 65.0)
print(mom)

In [ ]:
def calc_powers(n):
    return (n**1, n**2, n**3, n**4, n**5)

calc_powers(2)

#### Scope

Variables are not explicitly declared in Python but there are different levels of scope with a program.  The idea of *scope* is the names of variables that are accessible at a particular point in a program.  Have some variables in the *global scope* can be useful for defining program wide parameters and constants. Hopefully, a few examples are sufficient to get the idea across.

In [ ]:
# global scope
a = 6
b = 42

def func(x, y):
    # local scope
    z = 16
    return a*x + b*y + z

# global scope
c = func(1, 5)

In [ ]:
a = 6

def a_global():
    print(a)

def a_local():
    a = 42
    print(a)

a_global()
a_local()
print(a)

*Advice:* In general don't worry too much about scope. Instead, try to ensure that you don't use the same variable name both as a global variable and a local variable.  Things may get confusing.

## Program structure

Now that we know how to use functions, here is the code rewritten with better program structure.

In [ ]:
# 1D radioactive decay

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

# initial number of uranium atoms
N_uranium_initial = 1000

# decay constant of 238 U 
k_per_s = 4.916e-18
# expressed as 'per year'
k = k_per_s * 365*24*60*60

In [ ]:
def solve(dt=100e6, tmax=10.0e9):
    # Given a time step dt, integrate a one decay process equation
    # up to a maximum time value of tmax.  
    # Returns arrays for time and number of atoms.
    
    # calculate an array for all times
    t = np.arange(0, tmax, dt)

    # determine total number of timesteps needed
    M = len(t)

    # initializes N_uranium, an array of dimension M to being all zeros
    N_uranium = np.zeros(M)

    # the initial condition, first entry in the array N_uranium is N_uranium_initial
    N_uranium[0] = N_uranium_initial

    # loop over the timesteps and calculate the numerical solution
    for i in range(1, M):
        N_uranium[i] = N_uranium[i-1] - (k * N_uranium[i-1])*dt

    return t, N_uranium

In [ ]:
def plot(t, N_uranium, style='', label=''):

    # Plot the solution 
    plt.plot(t, N_uranium, style, label=label); 
    
    # label axes
    plt.xlabel('Time in years')
    plt.ylabel('Number of atoms')

    # axes scales
    plt.xlim(0, max(t))
    plt.ylim(0, N_uranium_initial)
    plt.legend()

In [ ]:
# create a new empty figure for plots
fig, axes = plt.subplots()
    
# numerically solve and plot
dt = 100e6
t, N_uranium = solve(dt=dt)
plot(t, N_uranium, style='b-', label='Numerical')
    
# For comparison, calculate and plot the analytical solution
N_analytical = N_uranium_initial * np.exp(-k * t)
plot(t, N_analytical, style='r-', label='Analytical')

plt.show()

* What are the functions defined in this program?

* Trace the order of execution.

* What variables are passed as arguments?

* What are the global variables (variables in the global scope)?

## Better graphics

Let's also consider the final piece of advice given above:

5\. **Take the time to make the graphics presentable**. In almost all cases, numerical results should be presented graphically. The axes should be labeled clearly (including units, where appropriate) and parameter values given directly on the graph.

We can do even a better job at following this advice by including the parameters directly on the plot. 




In [ ]:
# create a string with k and dt embedded
parameters = 'Decay constant {:.2e}\nTime step {:.2e}'.format(k, dt)

# add this text to the axes of the plot
axes.text(0.1e10, 200, parameters, fontsize=12)

# show the same figure again in this cell
fig

## Testing

*Giordano 1.4*

Creating a working program is more than just getting the code to run without any syntax errors.  We also need to be concerned about whether the output is correct!  Checking a program is not always a trivial task but here are some guidelines:

1\. **Does the output look reasonable?** Before you perform any calculation you should always have at least a rough idea of what the result should be. The first thing you should do when considering the results from any program is ask whether or not they are consistent with your intuition and instincts. This exercise can also improve your overall understanding of the problem. When you show your result to someone else, you should always be able to convice them that it makes sense.

2\. **Does the program agree with any exact results that are available?**  Since we knew the analytical solution for our radioactive decay problem, we were able to compare our numerical values with the exact result. While such a comparison will not be possible for most of the numerical calculations you will encounter, exact results are sometime available in certain limits, that is, for special values of the paramters. You should always run your program in those limits to check that it gives the correct answer. This is a necessary (but not sufficient) test that a program is correct in the general case.

3\. **Always check that your program gives the same answer for different *step sizes*.** Our decay program involved a time-step variable, `dt`, and most other numerical calculation involve similar step- or grid-size parameters. Your final answer should be independent of the values of such parameters. This is another necessary (but not sufficient) test of a program's accuracy.

Checking a program should not be viewed as a trivial, last minute job. It is not unreasonable to spend as much time checking a program as it takes writing it. A result is not much good if you don't trust it to be correct.

### Changing the step size

*Giordano section 1.5*

You may be asking why we went to the effort of writting the code to have separate functions for the solving and the plotting. Were we not done with this problem with the version of the code we started with in the lecture?

We can use this added level of program structure to quickly investigate the impact of step size. Without this program structure, we might find ourselves copying and pasting large sections of our program.  It is good programming style to DRY (*Do not Repeat Yourself*).

Consider the following code:

In [ ]:
fig, axes = plt.subplots()

t, N_uranium = solve(dt=1e6)
plot(t, N_uranium, label='dt = 50e6', style=':')

t, N_uranium = solve(dt=100e6)
plot(t, N_uranium, label='dt = 100e6', style='-')

t, N_uranium = solve(dt=500e6)
plot(t, N_uranium, label='dt = 500e6', style='o-')
    
t, N_uranium = solve(dt=3000e6)
plot(t, N_uranium, label='dt = 3000e6', style='o-')

plt.title('Numerical error with different step sizes')
plt.show()

The solution depends on the value of step size, `dt`, we chose. For large values of dt the numerical solution under estimates the number of uranium atoms.  With successively smaller values of dt the numerical solution appears to be converging to solution.  You should always check the effect of time step on our numerical solutions.

Numerical errors are an unavoidable aspect of computational physics.  We will investigate the mathematics of how numerical errors arise briefly later on this course. In advanced courses on numerical analysis and scientific computation, you learn more about the numerical properities of algorithms.

#  Next steps

1. Distribute chapter 2 of Giordano and Nakanishi.  Lectures 3 - 6 are based on this chapter.  Please read at least before the start of Lab 2 next week.
2. Discuss Assignment 1.

<!--NAVIGATION-->
< [Solving Problems Numerically](Lecture.01-Solving-Problems-Numerically.ipynb) | [Contents](Index.ipynb) | [Bicycle Racing: The Effect of Air Resistance](Lecture.03-Bicyle-Racing.ipynb) >